In [1]:
from openai import AzureOpenAI
from openai import OpenAI

In [2]:
# api_version = "2024-07-18"
api_version = "2024-02-15-preview"
azure_endpoint="https://nshan-m3oer8jc-eastus.openai.azure.com/"
azureClient = AzureOpenAI(api_version=api_version, azure_endpoint=azure_endpoint)
# azureClient = AzureOpenAI(azure_endpoint=azure_endpoint)

In [3]:
q = 'the course has already started, can I still enroll?'
response = azureClient.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [4]:
response.choices[0].message.content

"To determine if you can still enroll in a course that has already started, you'll need to check with the specific institution or course provider offering the program. Here are a few general steps you can take:\n\n1. **Contact the Institution:** Reach out to the admissions office or the department offering the course to inquire about late enrollment policies. Some institutions may allow late enrollments within a certain timeframe.\n\n2. **Check the Course Website:** Some programs have specific guidelines regarding late enrollment, which might be listed on their website.\n\n3. **Consider Special Circumstances:** If you have a compelling reason for enrolling late, such as personal or professional obligations, explain your situation to the course provider. They may be more flexible given your circumstances.\n\n4. **Online Platforms:** If it's an online course, whether through a university or a platform like Coursera or edX, check their FAQ or help sections regarding late enrollments, as p